In [1]:
Experiments_file='/home/kate/Research/YearBuilt/Experiments/Experiments.xlsx'
#Experimets_tab: #,Experiment,Objective: Objective is the column name for desciption
Experiments_tab='Experiments'

In [2]:
import boto3
import time
import pandas as pd
import numpy as np
import sagemaker
from smexperiments.experiment import Experiment

from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

In [ ]:
import sys
sys.path.append('/home/kate/Research/YearBuilt/Notebooks/Property')
import ExperimentsUtils as eu

In [3]:
Experiments=pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiments_tab)

In [4]:
Experiments   

,#,Experiment,Objective,Status,Result,Dataset,DataPreparation Script,Model Training Script,Inference Script,DataPostProcessing Script
0,0.0,BaseFeatures,Compose a base set of features,Done,Only few models as examples were tested. Usual...,property_basedata_v3.csv,Create training and validation datasets - Feat...,Training Models - Features Research.ipynb,NaN,NaN
1,1.0,BinnedFeatures,Research impact binning features on accuracy...,Done,Best models are based on notbinned attributes....,property_basedata_v3.csv,Create training and validation datasets - Feat...,Training Models - Features Research.ipynb,NaN,NaN
2,2.0,YearBuiltvsAge,What's better for higher accuracy: YearBuilt o...,Done,"If there is a difference between models, it's ...",property_basedata_v3.csv,Create training and validation datasets - Feat...,Training Models - Features Research.ipynb,NaN,NaN
3,3.0,MultiPolicy,"Impact of LandlordInd, MultiPolicyInd or numbe...",Done,Adding Landlord or ActiveOtherPolicies definet...,property_basedata_v3.csv,Create training and validation datasets - Feat...,Training Models - Features Research.ipynb,NaN,NaN
4,3.1,BestModels,Comparing 3 best models from multiPolicy with ...,Done,t-Test confirmed there is no need to calculate...,property_basedata_v3.csv,Create training and validation datasets for t-...,Training Models and t-Test.ipynb,NaN,NaN
5,4.0,PropertyFeatures,"Impact of Water Score, ConstructionCd and Prop...",Done,There is no visible difference between differe...,property_basedata_v3.csv,Create training and validation datasets - Feat...,Training Models - Features Research.ipynb,NaN,NaN
6,4.1,BestModels2,The difference between models selected is very...,Done,According to t-Test there is no difference bet...,property_basedata_v3.csv,Create training and validation datasets for t-...,Training Models and t-Test.ipynb,NaN,NaN
7,5.0,Overfitting,To verify theer is no overfitting in a model,Done,"With more data in the training dataset, the mo...",property_basedata_v3.csv,Create training and validation datasets for ov...,Training Models and overfitting test.ipynb,NaN,NaN
8,6.0,Params,Basic parameters tuning. Just see a change in ...,Done,No significant improvement found,property_basedata_v3.csv,NaN,Training Models - Parameters Research.ipynb,NaN,NaN
9,7.0,PropertyHyperParams2,Deep parameters tuning at least for some time.,Done,No significant improvement found,property_basedata_v3.csv,NaN,Hyperparameter Tuning job.ipynb,NaN,NaN


In [5]:
#should be run as a first step
#role arn is used when run from a local machine
role = 'arn:aws:iam::757107622481:role/service-role/AmazonSageMaker-ExecutionRole-20200819T131882'
region = 'us-west-2'
sm_sess = sagemaker.session.Session()
sm = boto3.Session().client('sagemaker')

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [6]:
#Experiments summaries
from sagemaker.analytics import ExperimentAnalytics

In [7]:
experiments_l = list()
job_l = list()
job_type_l = list()
TransformingTimeInSeconds_l = list()
ProcessingTimeInSeconds_l = list()
BillableTimeInSeconds_l = list()
TrainingTimeInSeconds_l = list()
InstanceType_l = list()
InstanceCount_l = list()
for e in Experiments['Experiment']:
    print('Processing experiment: %s'%e)
    analytics = ExperimentAnalytics(experiment_name=e)
    analytics_ds = analytics.dataframe()
    if analytics_ds.empty:
        continue
    for j in analytics_ds['SourceArn'].values:
        experiments_l.append(e)
        job_name=j[j.index('/')+1:len(j)]
        job_l.append(job_name)
        if 'training-job' in j:
            job_type_l.append('training-job')
            job_desc=sm.describe_training_job(TrainingJobName=job_name)
            BillableTimeInSeconds_l.append(job_desc['BillableTimeInSeconds'])
            TrainingTimeInSeconds_l.append(job_desc['TrainingTimeInSeconds'])
            InstanceType_l.append(job_desc['ResourceConfig']['InstanceType'])
            InstanceCount_l.append(job_desc['ResourceConfig']['InstanceCount'])
            ProcessingTimeInSeconds_l.append(0)
            TransformingTimeInSeconds_l.append(0)            
        elif 'processing-job' in j:
            job_type_l.append('processing-job')        
            job_desc=sm.describe_processing_job(ProcessingJobName=job_name)
            duration = job_desc['ProcessingEndTime'] - job_desc['ProcessingStartTime']
            ProcessingTimeInSeconds_l.append(float(str(duration.seconds) +'.'+ str(duration.microseconds)))
            InstanceType_l.append(job_desc['ProcessingResources']['ClusterConfig']['InstanceType'])
            InstanceCount_l.append(job_desc['ProcessingResources']['ClusterConfig']['InstanceCount'])
            BillableTimeInSeconds_l.append(0)
            TrainingTimeInSeconds_l.append(0)      
            TransformingTimeInSeconds_l.append(0)
        elif 'transform-job' in j:
            job_type_l.append('transform-job')        
            job_desc=sm.describe_transform_job(TransformJobName=job_name)
            duration = job_desc['TransformEndTime'] - job_desc['TransformStartTime']
            TransformingTimeInSeconds_l.append(float(str(duration.seconds) +'.'+ str(duration.microseconds)))
            InstanceType_l.append(job_desc['TransformResources']['InstanceType'])
            InstanceCount_l.append(job_desc['TransformResources']['InstanceCount'])
            BillableTimeInSeconds_l.append(0)
            TrainingTimeInSeconds_l.append(0)  
            ProcessingTimeInSeconds_l.append(0)
JobsSummary = pd.DataFrame(list(zip(experiments_l, job_l, job_type_l, ProcessingTimeInSeconds_l, BillableTimeInSeconds_l, TrainingTimeInSeconds_l, TransformingTimeInSeconds_l, InstanceType_l, InstanceCount_l)), 
columns =['Experiment','Job Name', 'Job Type', 'ProcessingTimeInSeconds', 'BillableTimeInSeconds', 'TrainingTimeInSeconds', 'TransformingTimeInSeconds','InstanceType', 'InstanceCount'])
JobsSummary

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Processing experiment: BaseFeatures


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Processing experiment: BinnedFeatures


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


...

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Processing experiment: PartialDependency


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Processing experiment: FinalResults


,Experiment,Job Name,Job Type,ProcessingTimeInSeconds,BillableTimeInSeconds,TrainingTimeInSeconds,TransformingTimeInSeconds,InstanceType,InstanceCount
0,BaseFeatures,limitdeductible-2020-12-17-16-39-24,training-job,0.000,738,738,0.0,ml.c5.xlarge,1
1,BaseFeatures,yearbuilt-2020-12-17-16-39-26,training-job,0.000,557,557,0.0,ml.c5.xlarge,1
2,BaseFeatures,startmodel-2020-12-17-16-39-29,training-job,0.000,102,102,0.0,ml.c5.xlarge,1
3,BaseFeatures,sagemaker-scikit-learn-2020-12-17-16-24-49-691,processing-job,121.109,0,0,0.0,ml.t3.medium,1
4,BaseFeatures,sagemaker-scikit-learn-2020-12-17-16-24-47-575,processing-job,88.680,0,0,0.0,ml.t3.medium,1
...,...,...,...,...,...,...,...,...,...
203,FinalResults,construction-waterscore-7-2020-12-22-23-44-48-724,processing-job,119.181,0,0,0.0,ml.m5.xlarge,1
204,FinalResults,construction-waterscore-6-2020-12-22-23-44-47-433,processing-job,114.302,0,0,0.0,ml.m5.xlarge,1
205,FinalResults,construction-waterscore-3-2020-12-22-23-44-52-482,processing-job,89.108,0,0,0.0,ml.m5.xlarge,1
206,FinalResults,construction-waterscore-8-2020-12-22-23-44-53-713,processing-job,85.775,0,0,0.0,ml.m5.xlarge,1


In [8]:
#Saving into the Experiment log 
eu.SaveToExperimentLog(Experiments_file, 'Experiments Cost', JobsSummary)